# All about Tensors
## Explaining Tensors Using a CNN Example

- [?, color, height, width]
- [3, 1, 28, 28] -> 3 images, each with one color scheme (greyscale), each with a dim of 28*28
- The tensors change shape as they propagate through the networks so it is important to reshape them correctly 
    - Eg. color channels -> [filter] -> feature maps
- Data must be passed through pre-processing routines to transform it to efficient tensor formats
- PyTorch Tensors have some specific attributes
    - dtype: data type of tensor float32, etc.
    - devcie: device on tensor
    - layout: tells about how tensors are laid out on memory
- Tensor computations must be of the same data type (i.e int + int = okay), and also must be on the same device

In [1]:
import torch
import torch.nn as nn
import numpy as np

t = torch.Tensor()
print(t.dtype)
print(t.device)
print(t.layout)

torch.float32
cpu
torch.strided


## What are the best options of creating PyTorch Tensors?

- PyTorch has several ways which we can use to create tensors
- All functions (except for the constructor method torch.Tensor()) allow us to set a dtype for our data
- Some functions allow you to modify arrays without influencing tensors and viceversa
    - zero memory-copy -> very efficient
        - torch.as_tensor() -> go to function of choice for this category
        - torch.from_numpy() -> only accepts numpy arrays
    - Simply copying data
        - torch.tensor() -> go to function of choice
        - torch.Tensory()

In [6]:
data = np.array([1,2,3])
# class constructor, uses global default dtype -> float32
t1 = torch.Tensor(data)
# factory function, OOP
t2 = torch.tensor(data)
# factory function
t3 = torch.as_tensor(data)
# factory function
t4 = torch.from_numpy(data)
print(t1.dtype, t2.dtype, t3.dtype, t4.dtype, sep='\n')

torch.float32
torch.int64
torch.int64
torch.int64


## Tensor Operations

- Reshaping Operations
    - Mold and modify tensors into the shape you desire
    - You can reshape a tensor as many ways as there are factors of the number of elements
        - i.e. 12 elements -> (12 = 12*1; 6*2; 4*3) 6 reshapes (order matters too)
    - Another way of reshaping is to squeeze and then unsqueeze to the desired tensor
- Reduction Operations
- Access Operations

- Tensors need to be flattened before they are passed to a fully connected layer in a CNN
    - My chromatograms are already flat, perhaps the baseline can be viewd as another channel
    
- In a CNN batch, you want to flatten each image in the batch because you need a prediction per each image
    - <code> t.flatten(start_dim=1) <code>  
        - <code> t.flatten(start_dim=1).shape <code> 
        eg. [3, 16] 3 images with 16 elements each
    - <code> t.reshape(t.shape[0],-1)



In [17]:
t = torch.tensor([
    [1, 1, 1, 1],
    [2, 2, 2, 2],
    [3, 3, 3, 3]
], dtype=torch.float32)
print(
    t.size(),
    t.shape,
    len(t.shape), # rank of tensor
    torch.tensor(t.shape).prod(), # number of elements
    t.numel(), # number of elements
sep='\n')

torch.Size([3, 4])
torch.Size([3, 4])
2
tensor(12)
12


In [25]:
# reshapes
print(
     t,
     t.reshape(1,12),
     #t.reshape(2,6), 
     #t.reshape(3,4),
     t.reshape(4,3),
     #t.reshape(6,2),
     #t.reshape(12,1),
sep='\n')

# flattening tensors
# useful for fully connected NN after convolutional layers
# the way it works is implemented as follows:
def flatten(t):
    t = t.reshape(1, -1)
    t = t.squeeze()
    return t
print(t.flatten()) # 1d array containing all scalars
#print(flatten(t))


tensor([[1., 1., 1., 1.],
        [2., 2., 2., 2.],
        [3., 3., 3., 3.]])
tensor([[1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.]])
tensor([[1., 1., 1.],
        [1., 2., 2.],
        [2., 2., 3.],
        [3., 3., 3.]])
tensor([1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.])
tensor([1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.])


In [38]:
# batch and tensors
t1 = torch.tensor([
    [1, 1, 1],
    [1, 1, 1], 
    [1, 1, 1]
])

t2 = torch.tensor([
    [2, 2, 2],
    [2, 2, 2], 
    [2, 2, 2]
])

t3 = torch.tensor([
    [3, 3, 3],
    [3, 3, 3], 
    [3, 3, 3]
])

t = torch.stack((t1,t2,t3))
print(t)
print(t.shape, '-> batch = 3')

print()
print('Reshaped tensor \n', t.reshape(3, 1, 3, 3))


tensor([[[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],

        [[2, 2, 2],
         [2, 2, 2],
         [2, 2, 2]],

        [[3, 3, 3],
         [3, 3, 3],
         [3, 3, 3]]])
torch.Size([3, 3, 3]) -> batch = 3

Reshaped tensor 
 tensor([[[[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]]],


        [[[2, 2, 2],
          [2, 2, 2],
          [2, 2, 2]]],


        [[[3, 3, 3],
          [3, 3, 3],
          [3, 3, 3]]]])


### Element-wise Operations

- Operates on tensor elements that correspond on index location 
- The tensors must have the same shape and number of elements
    - Otherwise the operation will make use of broadcasting
    - It is important to understand broadcasting, it will help you avoid writing unnecessary loops!

In [45]:
t1 = torch.tensor([
    [1, 2],
    [3,4]
])

t2 = torch.tensor([
    [9,8],
    [7,6]
])

print(t1[0], t1[0][0], sep='\n')
print(t2[0], t1[0][0], sep='\n')
print()
print(t1+t2)
print(t2-1)

print(
    t1 + torch.tensor(
    np.broadcast_to(2, t1.shape),
    dtype=torch.float32)
)


tensor([1, 2])
tensor(1)
tensor([9, 8])
tensor(1)

tensor([[10, 10],
        [10, 10]])
tensor([[8, 7],
        [6, 5]])
tensor([[3., 4.],
        [5., 6.]])


### Tensor Reduction Operations

- An operation that reduces the number of elements contained in a tensor
    - sum, prod, mean, std etc.
    - It is also possible to do these operations on a certain dimension 
    - Argmax -> max output value of a function (the index location)
        - We use the argmax function on the output of a NN

In [20]:
t = torch.tensor([
    [0,1,0],
    [2,0,2],
    [0,3,0]
], dtype=torch.float32)

print(t, t.sum(), t.numel(), t.sum().numel(), sep='\n')
print(t.sum(dim=0)) # column sum
print(t.sum(dim=1)) # row sum
print('\nMax value: ', t.max(),' \nLocation: ', t.argmax()) # index 7
print()
print(t.max(dim=0))
print(t.max(dim=1))
print()
print(t.mean())
print(t.mean(dim=0).numpy())

tensor([[0., 1., 0.],
        [2., 0., 2.],
        [0., 3., 0.]])
tensor(8.)
9
1
tensor([2., 4., 2.])
tensor([1., 4., 3.])

Max value:  tensor(3.)  
Location:  tensor(7)

torch.return_types.max(
values=tensor([2., 3., 2.]),
indices=tensor([1, 2, 1]))
torch.return_types.max(
values=tensor([1., 2., 3.]),
indices=tensor([1, 0, 1]))

tensor(0.8889)
[0.6666667 1.3333334 0.6666667]
